In [16]:
import numpy as np
import random as rnd
import os
import re
from IPython.display import HTML, display
import matplotlib.pyplot as plt

In [17]:
languages = ['java', 'graal', 'nodejs', 'go', 'posix']
modes = ['base', 'static', 'dynamic']
fig_counter = 0
root='../../../../target/logs'
models='../../../../target/models'
sources='../../../../target/code'

In [18]:
def allFiles(language, mode, log):
    directory = os.fsencode(root + '/' + language + '/' + mode)
    for file in os.listdir(directory):
        filename = root + '/' + language + '/' + mode + '/' + os.fsdecode(file)
        if log:            
            if filename.split('/')[-1].startswith('nolog'+language) and filename.endswith('.log'): 
                yield filename
        else:
            if filename.split('/')[-1].startswith(language) and filename.endswith('.log'): 
                yield filename

def isWrong(filename):
    file = open(filename)
    score = -1
    while True:
        line = file.readline()      
        if not line:
            if score != 349:
                return 1
            else:
                return 0
            break        
        if line.startswith('ts:'):            
            score = int(line[1:].strip().split(',')[2].split(':')[1:][0])
            
def loadMemory(filename):
    file = open(filename)
    mem_sum = 0
    mem_count = 0
    while True:
        line = file.readline()      
        if not line:
            return mem_sum/mem_count        
        if line.startswith('#usedMem:'):
            mem = int(line[1:].strip().split(':')[1:][0])
            mem_sum += mem
            mem_count += 1
            
def loadTotalTime(filename):
    file = open(filename)
    while True:
        line = file.readline()      
        if not line:
            break        
        if line.startswith('>done in '):
            time = line[9:-4].strip()
            return int(time)              

In [31]:
def allData(language, mode):
    all_errors = []
    all_mem = []
    all_t = []
    for filename in allFiles(language, mode, True):
        e = isWrong(filename)
        if not e:
            mem = loadMemory(filename)
            t = loadTotalTime(filename)
            all_mem.append(mem)
            all_t.append(t)
        all_errors.append(e)        
    return all_t, all_mem, all_errors

In [41]:
for language in languages:   
    print(language)
    print('-------')
    for mode in modes:
        print('in ' + mode + ' mode')
        t, mem, errs = allData(language, mode)
        print('\t-total errs: ' + str(sum(errs)) + ' / ' + str(len(errs)) + ' (' + str(sum(errs)/len(errs)) + ')')
        if len(t) > 0 :
            print('\t-avg time:   ' + str(sum(t)/len(t)))
        if len(mem) > 0 :            
            print('\t-avg mem:    ' + str(sum(mem)/len(mem)))        
    print()

java
-------
in base mode
	-total errs: 0 / 3 (0.0)
	-avg time:   6458.333333333333
	-avg mem:    644491.2380952381
in static mode
	-total errs: 0 / 3 (0.0)
	-avg time:   6497.666666666667
	-avg mem:    695917.3968253968
in dynamic mode
	-total errs: 0 / 3 (0.0)
	-avg time:   6680.333333333333
	-avg mem:    744363.4285714285

graal
-------
in base mode
	-total errs: 3 / 3 (1.0)
in static mode
	-total errs: 3 / 3 (1.0)
in dynamic mode
	-total errs: 3 / 3 (1.0)

nodejs
-------
in base mode
	-total errs: 0 / 3 (0.0)
	-avg time:   9587.666666666666
	-avg mem:    5165093.142857143
in static mode
	-total errs: 0 / 3 (0.0)
	-avg time:   9985.333333333334
	-avg mem:    5360287.492063493
in dynamic mode
	-total errs: 0 / 3 (0.0)
	-avg time:   10060.666666666666
	-avg mem:    5738417.8412698405

go
-------
in base mode
	-total errs: 0 / 3 (0.0)
	-avg time:   6166.333333333333
	-avg mem:    129398.66666666667
in static mode
	-total errs: 0 / 3 (0.0)
	-avg time:   5907.666666666667
	-avg mem:    1